# Question 1

# B

In [34]:
import gurobipy as gp
from gurobipy import GRB

# Sets of suppliers and plants
suppliers = [1, 2, 3]
plants = [1, 2, 3]

# Supply availability at each supplier
supply = {1: 50, 2: 120, 3: 180}

# Demand requirements at each plant
demand = {1: 80, 2: 90, 3: 60}

# Transportation costs from each supplier to each plant
cost = {
    (1, 1): 320, (1, 2): 215, (1, 3): 400,
    (2, 1): 410, (2, 2): 245, (2, 3): 360,
    (3, 1): 290, (3, 2): 460, (3, 3): 190
}

# Create the model
model = gp.Model("TransportationProblem")

# Decision variables: x[s, p] is the amount shipped from supplier s to plant p
x = model.addVars(suppliers, plants, name="x", vtype=GRB.CONTINUOUS, lb=0)


# Objective: Minimize total transportation cost
model.setObjective(gp.quicksum(cost[s, p] * x[s, p] for s in suppliers for p in plants), GRB.MINIMIZE)

# Supply constraints: The total amount shipped from each supplier cannot exceed its supply
for s in suppliers:
    model.addConstr(gp.quicksum(x[s, p] for p in plants) <= supply[s], name=f"supply_{s}")

# Demand constraints: Each plant must receive exactly the amount it requires
for p in plants:
    model.addConstr(gp.quicksum(x[s, p] for s in suppliers) == demand[p], name=f"demand_{p}")

# Solve the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal transportation strategy:")
    for s in suppliers:
        for p in plants:
            if x[s, p].x > 0:
                print(f"Ship {x[s, p].x:.2f} tons from Supplier {s} to Plant {p} at cost {cost[s, p]} per ton")

    print(f"\nTotal transportation cost: {model.objVal:.2f}")
else:
    print("No optimal solution found.")

{(1, 1): <gurobi.Var *Awaiting Model Update*>, (1, 2): <gurobi.Var *Awaiting Model Update*>, (1, 3): <gurobi.Var *Awaiting Model Update*>, (2, 1): <gurobi.Var *Awaiting Model Update*>, (2, 2): <gurobi.Var *Awaiting Model Update*>, (2, 3): <gurobi.Var *Awaiting Model Update*>, (3, 1): <gurobi.Var *Awaiting Model Update*>, (3, 2): <gurobi.Var *Awaiting Model Update*>, (3, 3): <gurobi.Var *Awaiting Model Update*>}
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xa3ce63a2
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+01, 2e+02]
Presolve time: 0.01s
Presolved: 6 rows, 9 columns, 18 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.3950000e+04   4

# C

In [36]:
import gurobipy as gp
from gurobipy import GRB

# Sets of suppliers and plants
suppliers = [1, 2, 3]
plants = [1, 2, 3]

# Supply availability after disruptions (adjusted supply)
supply = {1: 50*(1-0.4), 2: 120*(1-0.6), 3: 180*(1-0.25)}


# Demand requirements at each plant
demand = {1: 80, 2: 90, 3: 60}

# Transportation costs from each supplier to each plant
cost = {
    (1, 1): 320, (1, 2): 215, (1, 3): 400,
    (2, 1): 410, (2, 2): 245, (2, 3): 360,
    (3, 1): 290, (3, 2): 460, (3, 3): 190
}

# Create the model
model = gp.Model("TransportationProblem_Disrupted")

# Decision variables: x[s, p] is the amount shipped from supplier s to plant p
x = model.addVars(suppliers, plants, name="x", vtype=GRB.CONTINUOUS, lb=0)

# Objective: Minimize total transportation cost
model.setObjective(gp.quicksum(cost[s, p] * x[s, p] for s in suppliers for p in plants), GRB.MINIMIZE)

# Supply constraints: The total amount shipped from each supplier cannot exceed its (disrupted) supply
for s in suppliers:
    model.addConstr(gp.quicksum(x[s, p] for p in plants) <= supply[s], name=f"supply_{s}")

# Demand constraints: Each plant must receive exactly the amount it requires
for p in plants:
    model.addConstr(gp.quicksum(x[s, p] for s in suppliers) == demand[p], name=f"demand_{p}")

# Solve the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal transportation strategy (with supply disruptions):")
    for s in suppliers:
        for p in plants:
            if x[s, p].x > 0:
                print(f"Ship {x[s, p].x:.2f} tons from Supplier {s} to Plant {p} at cost {cost[s, p]} per ton")

    print(f"\nTotal transportation cost: {model.objVal:.2f}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 9 columns and 18 nonzeros
Model fingerprint: 0xe6cac65d
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+02, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+02]
Presolve removed 2 rows and 4 columns
Presolve time: 0.00s
Presolved: 4 rows, 5 columns, 10 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.6930000e+04   1.500000e+01   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Infeasible model
No optimal solution found.


# D

In [43]:
import gurobipy as gp
from gurobipy import GRB

# Sets of suppliers and plants
suppliers = [1, 2, 3]
plants = [1, 2, 3]

# Supply availability at each supplier
supply = {1: 50, 2: 120, 3: 180}

# Demand requirements at each plant
demand = {1: 80, 2: 90, 3: 60}

# Transportation costs from each supplier to each plant
cost_supplier_to_plant = {
    (1, 1): 320, (1, 2): 215, (1, 3): 400,
    (2, 1): 410, (2, 2): 245, (2, 3): 360,
    (3, 1): 290, (3, 2): 460, (3, 3): 190
}

# Transportation costs between plants
cost_plant_to_plant = {
    (1, 2): 15, (1, 3): 20,
    (2, 1): 30, (2, 3): 25,
    (3, 1): 20, (3, 2): 15
}

# Create the model
model = gp.Model("TransportationProblemWithSingleInternalShipping")

# Decision variables: x[s, p] is the amount shipped from supplier s to plant p
x = model.addVars(suppliers, plants, name="x", vtype=GRB.CONTINUOUS, lb=0)

# Decision variables: y[p, q] is the amount shipped from plant p to plant q
y = model.addVars(plants, plants, name="y", vtype=GRB.CONTINUOUS, lb=0)

# Binary decision variables: z[p] is 1 if plant p ships to another plant, 0 otherwise
z = model.addVars(plants, vtype=GRB.BINARY)

# Binary variables: w[p, q] is 1 if plant p ships to plant q, 0 otherwise
w = model.addVars(plants, plants, vtype=GRB.BINARY)

# Objective: Minimize total transportation cost (from suppliers and between plants)
model.setObjective(
    gp.quicksum(cost_supplier_to_plant[s, p] * x[s, p] for s in suppliers for p in plants) +
    gp.quicksum(cost_plant_to_plant[p, q] * y[p, q] for p in plants for q in plants if p != q), 
    GRB.MINIMIZE)

# Supply constraints: The total amount shipped from each supplier cannot exceed its supply
for s in suppliers:
    model.addConstr(gp.quicksum(x[s, p] for p in plants) <= supply[s], name=f"supply_{s}")

# Demand constraints: Each plant must receive exactly the amount it requires (from suppliers + internal)
for p in plants:
    model.addConstr(
        gp.quicksum(x[s, p] for s in suppliers) + gp.quicksum(y[q, p] for q in plants if q != p) == demand[p], 
        name=f"demand_{p}")

# Internal shipping constraint: Only one plant can ship to exactly one other plant
model.addConstr(gp.quicksum(z[p] for p in plants) == 1, name="one_plant_ships")

# Ensure that y[p, q] is only positive if plant p is allowed to ship (z[p] = 1) and only to one other plant (w[p, q] = 1)
for p in plants:
    for q in plants:
        if p != q:
            model.addConstr(y[p, q] <= 1000 * w[p, q], name=f"link_y_to_w_{p}_{q}")
            model.addConstr(w[p, q] <= z[p], name=f"link_w_to_z_{p}")

# Ensure that each selected plant can only ship to one other plant
for p in plants:
    model.addConstr(gp.quicksum(w[p, q] for q in plants if q != p) <= 1, name=f"one_destination_{p}")

# Solve the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    print("Optimal transportation strategy (with restricted internal shipping):")
    for s in suppliers:
        for p in plants:
            if x[s, p].x > 0:
                print(f"Ship {x[s, p].x:.2f} tons from Supplier {s} to Plant {p} at cost {cost_supplier_to_plant[s, p]} per ton")
    for p in plants:
        for q in plants:
            if p != q and y[p, q].x > 0:
                print(f"Ship {y[p, q].x:.2f} tons from Plant {p} to Plant {q} at cost {cost_plant_to_plant[p, q]} per ton")

    print(f"\nTotal transportation cost: {model.objVal:.2f}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 30 columns and 57 nonzeros
Model fingerprint: 0xf92d9f6f
Variable types: 18 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Found heuristic solution: objective 73950.000000
Presolve removed 10 rows and 10 columns
Presolve time: 0.00s
Presolved: 12 rows, 20 columns, 40 nonzeros
Variable types: 15 continuous, 5 integer (5 binary)

Root relaxation: objective 3.355000e+04, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    33550.000000 33550.0000  0.00%     -   

# Question 2

# B

In [27]:
import gurobipy as gp
from gurobipy import GRB

# Define sets
hubs = [1, 2, 3]
warehouses = [1, 2, 3]
customers = [1, 2, 3, 4, 5]
sauce_types = ['r', 's']

# Define parameters
cost_hub_to_warehouse = {(1, 1): 100, (1, 2): 370, (1, 3): 90,
                         (2, 1): 225, (2, 2): 450, (2, 3): 300,
                         (3, 1): 150, (3, 2): 400, (3, 3): 200}

cost_warehouse_to_customer = {(1, 1): 50, (1, 2): 70, (1, 3): 30, (1, 4): 25, (1, 5): 50,
                              (2, 1): 60, (2, 2): 40, (2, 3): 75, (2, 4): 40, (2, 5): 75,
                              (3, 1): 50, (3, 2): 35, (3, 3): 30, (3, 4): 80, (3, 5): 50}

unmet_demand_cost = {(1, 'r'): 850, (2, 'r'): 600, (3, 'r'): 950, (4, 'r'): 150, (5, 'r'): 50,
                     (1, 's'): 250, (2, 's'): 600, (3, 's'): 50, (4, 's'): 650, (5, 's'): 1000}

demand = {(1, 'r'): 200, (2, 'r'): 250, (3, 'r'): 150, (4, 'r'): 100, (5, 'r'): 200,
          (1, 's'): 100, (2, 's'): 50, (3, 's'): 100, (4, 's'): 200, (5, 's'): 350}

max_orders_hub_to_warehouse = {(1, 1): 50, (1, 2): 30, (1, 3): 60,
                               (2, 1): 40, (2, 2): 35, (2, 3): 85,
                               (3, 1): 45, (3, 2): 50, (3, 3): 55}

max_orders_warehouse_to_customer = {(1, 1): 40, (1, 2): 35, (1, 3): 25, (1, 4): 50, (1, 5): 25,
                                    (2, 1): 45, (2, 2): 35, (2, 3): 25, (2, 4): 10, (2, 5): 60,
                                    (3, 1): 65, (3, 2): 30, (3, 3): 40, (3, 4): 20, (3, 5): 50}

# Create a new model
m = gp.Model('supply_chain')

# Decision variables
x = m.addVars(hubs, warehouses, sauce_types, vtype=GRB.INTEGER, name="x")  # Number of orders from hubs to warehouses
y = m.addVars(warehouses, customers, sauce_types, vtype=GRB.INTEGER, name="y")  # Number of orders from warehouses to customers
u = m.addVars(customers, sauce_types, vtype=GRB.INTEGER, name="u")  # Unmet demand

# Objective function
m.setObjective(
    gp.quicksum(cost_hub_to_warehouse[h, w] * x[h, w, t] for h in hubs for w in warehouses for t in sauce_types) +
    gp.quicksum(cost_warehouse_to_customer[w, c] * y[w, c, t] for w in warehouses for c in customers for t in sauce_types) +
    gp.quicksum(unmet_demand_cost[c, t] * u[c, t] for c in customers for t in sauce_types),
    GRB.MINIMIZE)

# Constraints
# Demand fulfillment
m.addConstrs((gp.quicksum(y[w, c, t] for w in warehouses) + u[c, t] == demand[c, t] for c in customers for t in sauce_types), "demand")

# Hub to warehouse flow limit
m.addConstrs((gp.quicksum(x[h, w, t] for t in sauce_types) <= max_orders_hub_to_warehouse[h, w] for h in hubs for w in warehouses), "hub_flow_limit")

# Warehouse to customer flow limit
m.addConstrs((gp.quicksum(y[w, c, t] for t in sauce_types) <= max_orders_warehouse_to_customer[w, c] for w in warehouses for c in customers), "warehouse_flow_limit")

# Flow conservation constraint: shipments from hubs to warehouses must equal shipments from warehouses to customers
m.addConstrs((gp.quicksum(x[h, w, t] for h in hubs) == gp.quicksum(y[w, c, t] for c in customers) for w in warehouses for t in sauce_types), "flow_conservation")

# Non-negativity constraint
m.addConstrs((x[h, w, t] >= 0 for h in hubs for w in warehouses for t in sauce_types), "x_nonnegativity")
m.addConstrs((y[w, c, t] >= 0 for w in warehouses for c in customers for t in sauce_types), "y_nonnegativity")
m.addConstrs((u[c, t] >= 0 for c in customers for t in sauce_types), "u_nonnegativity")

# Optimize model
m.optimize()

# Retrieve and display results in the desired format
print("Orders shipped from hubs to warehouses:")
for h in hubs:
    for w in warehouses:
        for t in sauce_types:
            if x[h, w, t].x > 0:
                print(f"Orders shipped from hub {h} to warehouse {w} for sauce {t}: {x[h, w, t].x:.1f}")

print("\nOrders shipped from warehouses to customers:")
for w in warehouses:
    for c in customers:
        for t in sauce_types:
            if y[w, c, t].x > 0:
                print(f"Orders shipped from warehouse {w} to customer {c} for sauce {t}: {y[w, c, t].x:.1f}")

print("\nUnmet demand for customers:")
for c in customers:
    for t in sauce_types:
        if u[c, t].x > 0:
            print(f"Unmet demand for customer {c} for sauce {t}: {u[c, t].x:.1f}")

print(f"\nTotal cost: {m.objVal:.1f}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.6.0 23G93)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 98 rows, 58 columns and 194 nonzeros
Model fingerprint: 0x2e9ef861
Variable types: 0 continuous, 58 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 1e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 4e+02]
Found heuristic solution: objective 1027500.0000
Presolve removed 66 rows and 10 columns
Presolve time: 0.00s
Presolved: 32 rows, 48 columns, 102 nonzeros
Variable types: 0 continuous, 48 integer (0 binary)
Found heuristic solution: objective 1019020.0000

Root relaxation: objective 7.672250e+05, 24 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0         